In [3]:
from Mysql import *

pyMysql(host="1.15.90.134", user="gtrepublic", passwd="123")
myCursor = pyMysql.createCursor()
my
mydb = mysql.connector.connect(
    host="1.15.90.134",
    user="gtrepublic",
    passwd="123",
    database="stock-data")
c = mydb.cursor()


In [11]:
c.execute('''CREATE TABLE SZ000002
( ID      INT PRIMARAY KEY  NOT NULL,
  TIME    TEXT              NOT NULL,
  CODE    TEXT              NOT NULL,
  HIGH    REAL                      ,
  LOW     REAL                      ,
  CLOSE   REAL                      ,
  OPEN    REAL                      ,
  PRECLOSE REAL                     ,
  VOLUME  REAL                      ,
  AMOUNT  REAL                      ,
  ADJUSTFLAG INT                    ,
  TURN    REAL                      ,
  TRADESTATUS INT                   ,
  PCTCHG  REAL                      ,
  PRTTM   REAL                      ,
  PSTTM   REAL                      ,
  PCFNCFTTM REAL                    ,
  PBMRQ   REAL                      ,
  ISST    INT                       ,
  DESCRIPTION CHAR(50));''')
mydb.commit()
print(2)


2


In [ ]:
import baostock as bs
import pandas as pd

def fetchBaostockOnline(stockCode):
    #### 登陆系统 ####
    lg = bs.login()
    # 显示登陆返回信息
    print('login respond error_code:' + lg.error_code)
    print('login respond  error_msg:' + lg.error_msg)

    #### 获取沪深A股历史K线数据 ####
    # 详细指标参数，参见“历史行情指标参数”章节；“分钟线”参数与“日线”参数不同。
    # 分钟线指标：date,time,code,open,high,low,close,volume,amount,adjustflag
    rs = bs.query_history_k_data_plus(
        stockCode,
        "date,code,open,high,low,close,preclose,volume,\
        amount,adjustflag,turn,tradestatus,pctChg,isST",  # 输出字段列表
        start_date='2000-01-01',
        end_date='2020-07-13',
        frequency="d",  # 日线
        adjustflag="1")  # 复权类型，默认不复权：3；1：后复权；2：前复权
    print('query_history_k_data_plus respond error_code:' + rs.error_code)
    print('query_history_k_data_plus respond  error_msg:' + rs.error_msg)

    #### 获得结果集dataframe ####
    data = rs.get_data()
    # 将字符串数值列的类型转成数值型
    data = data.apply(pd.to_numeric, axis=0, errors='ignore')

    # 删除股票停牌的行记录
    data = data[data.tradestatus == 1]  # tradestatus 1 正常交易， 0 停牌
    # 日期列的类型
    data['date'] = pd.to_datetime(data['date'])
    print(data.head())
    return data

def uploadBaostockToDb(df_data, stockCode, conn):
    df_data.to_sql(name=stockCode, con=conn, index=False, if_exists='replace')

In [ ]:
df_data = fetchBaostockOnline("sh.600000")
uploadBaostockToDb(df_data, "SZ000002", mydb)
    

In [6]:
df_data = fetchBaostockOnline("sh.600000")
uploadBaostockToDb(df_data, "SZ000002", mydb)
    

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
        date       code   open   high    low  close  preclose    volume  \
0 2000-01-04  sh.600000  24.98  25.78  24.75  25.57     24.75   4496068   
1 2000-01-05  sh.600000  25.57  25.98  25.15  25.28     25.57   5252814   
2 2000-01-06  sh.600000  25.18  26.30  25.05  25.99     25.28   6229651   
3 2000-01-07  sh.600000  26.30  27.50  26.12  26.90     25.99  21355330   
4 2000-01-10  sh.600000  27.00  27.85  26.71  27.25     26.90  16539731   

        amount  adjustflag      turn  tradestatus    pctChg  isST  
0  113946783.0           1  1.405021            1  3.313130     0  
1  134465427.0           1  1.641504            1 -1.134138     0  
2  160059798.0           1  1.946766            1  2.808541     0  
3  575751082.0           1  6.673541            1  3.501346     0  
4  450453518.0           1  5.1686

DatabaseError: Execution failed on sql 'SELECT name FROM sqlite_master WHERE type='table' AND name=?;': Not all parameters were used in the SQL statement